## Class Announcements
- **Project Proposal** due Friday
    - submit in your group's repo on GitHub
    - use the Project Proposal document in Projects repo
    - you cloned that repo onto your computer for A1
    - one submission per group
- **Week 3 quiz** available (due Friday)
- **A2** due next Friday
- **Office Hours (Prof Ellis)** - 2:30 - 4 PM *this week only*

### Final Group Repos: Created on GitHub 
- Incorrect PID: 6 people
- Incorrect GH username: 5 people (dmunajat, esuaestrada, rkuijnj, harbor_s, poonamvarkhedi)
- Group sizes: 1 group of 3 people
- Students without a group: 43

# Data Wrangling, Data Intuition & Wisdom of the Crowds

Putting it all together:
- pandas
- data wrangling
- data inuition
- wisdom of the crowds

We'll use your responses to the Fermi estimation Google Form to do this.

## Setup

In [1]:
# import working with data libraries
import pandas as pd
import numpy as np

## The Data

In [2]:
# read data into Python
df = pd.read_csv('data/fermi_wi20.csv')

In [3]:
# take a look at the data
df

,Timestamp,What is your height (in cm)?,What is the average height of the people in this room (in cm)?,How fast does human hair grow (cm/yr)?,"If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?","How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?"
0,1/23/2020 9:18:26,150,170,10,6000,lots
1,1/23/2020 9:18:32,150,125,10,10000,200
2,1/23/2020 9:18:40,173.5,168,8,1000000,22
3,1/23/2020 9:18:54,178cm,169cm,15cm/yr,0.2,90
4,1/23/2020 9:18:57,155,140,36,12,365
...,...,...,...,...,...,...
71,1/23/2020 11:35:30,177.8,160,15,4000,4 months
72,1/23/2020 11:38:26,185,168,30,1000,100
73,1/23/2020 11:39:13,170.18,168.91,15.24,2723000000,38
74,1/23/2020 11:48:00,180.34,172.72,15.24,"285,750 (km^2)",40


In [4]:
# how does pandas handle commas?
df.iloc[39,:]

Timestamp                                                                                                                                                                   1/23/2020 9:24:41
What is your height (in cm)?                                                                                                                                                              165
What is the average height of the people in this room (in cm)?                                                                                                                            180
How fast does human hair grow (cm/yr)?                                                                                                                                                     20
If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?                                                                         290,000,000
How many days would it take to walk from here to N

## Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### iClicker Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

## Brainstorming

What considerations do we have to make about these data?

- 

In [5]:
# change column names
df.columns = ['timestamp','height_ind', 'height_class', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

,timestamp,height_ind,height_class,hair_growth,crammed,SAN_NYC
0,1/23/2020 9:18:26,150,170,10,6000,lots
1,1/23/2020 9:18:32,150,125,10,10000,200
2,1/23/2020 9:18:40,173.5,168,8,1000000,22
3,1/23/2020 9:18:54,178cm,169cm,15cm/yr,0.2,90
4,1/23/2020 9:18:57,155,140,36,12,365


In [6]:
# check type of each Series (column)
df.dtypes

timestamp       object
height_ind      object
height_class    object
hair_growth     object
crammed         object
SAN_NYC         object
dtype: object

In [ ]:
# this cell will produce an error, have to clean data first 
# df['height_ind'].median()

## Cleaning: Height

- Remove responses that included unit in response
- Apply to both height Series (columns)

Questions: 
- What is your height (in cm)?
- What is the average height of the people in this room (in cm)?

In [7]:
# take a look at unique values
df["height_ind"].unique()

array(['150', '173.5', '178cm', '155', '158', '165', '166', '176', '178',
       '170.1', '182', '160', '120', '170', '162', '175', '162.56', '180',
       '62', '157', '183', '190', '167', '60', '168', '171', '210', '172',
       '142 cm ', '157 cm', '154.94', '173', '185', '200', '187', '184',
       '164', '159', '169', '177.8', '170.18', '180.34', '163'],
      dtype=object)

In [8]:
# take a look at unique values
df["height_class"].unique()

array(['170', '125', '168', '169cm', '140', '173', '178', '180', '160',
       '175', '200', '130', '1x10^2 cm', '172', '164', '65', '165', '176',
       '150', '60', '165 cm ', '162', '162.25', '165cm', '170.18',
       '168.91', '172.72'], dtype=object)

In [9]:
# standardize height column
def standardize_height(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of inclded unit cases
    string = string.replace("cm", "")
    string = string.strip()
    
    # take care of scientific notation cases
    string = string.replace("x10^2", "00")

    # convert to numeric
    output = float(string)
    
    return output

In [10]:
# apply function across values in height columns
df["height_ind"] = df["height_ind"].apply(standardize_height)
df["height_class"] = df["height_class"].apply(standardize_height)

In [11]:
# check output after apply
df['height_ind'].unique()

array([150.  , 173.5 , 178.  , 155.  , 158.  , 165.  , 166.  , 176.  ,
       170.1 , 182.  , 160.  , 120.  , 170.  , 162.  , 175.  , 162.56,
       180.  ,  62.  , 157.  , 183.  , 190.  , 167.  ,  60.  , 168.  ,
       171.  , 210.  , 172.  , 142.  , 154.94, 173.  , 185.  , 200.  ,
       187.  , 184.  , 164.  , 159.  , 169.  , 177.8 , 170.18, 180.34,
       163.  ])

In [ ]:
# check output after apply
df['height_class'].unique()

In [12]:
df['height_ind'].median()

170.0

In [13]:
df['height_class'].median()

170.0

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

### iClicker Question #2

Which do you think is closest the actual speed at which hair grows (in cm/yr)?

- A) 1
- B) 10
- C) 100
- D) 1000

In [14]:
# take a look at unique values
df["hair_growth"].unique()

array(['10', '8', '15cm/yr', '36', '20', '4', '15', '20/', '30', '60',
       '12', '10cm/yr', '152', '25cm/yr', '100', '4/yr', '6', '50', '40',
       '17', '2 cm ', '5', '7', '1.5', '144 cm ', '60 cm/ye', '3', '300',
       '0.01', '9cm/yr', '15.24', '30cm/yr'], dtype=object)

In [15]:
# standardize height column
def standardize_hair(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of inclded unit cases
    string = string.replace("cm/ye", "")
    string = string.replace("cm/yr", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")

    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [16]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

array([1.000e+01, 8.000e+00, 1.500e+01, 3.600e+01, 2.000e+01, 4.000e+00,
       3.000e+01, 6.000e+01, 1.200e+01, 1.520e+02, 2.500e+01, 1.000e+02,
       6.000e+00, 5.000e+01, 4.000e+01, 1.700e+01, 2.000e+00, 5.000e+00,
       7.000e+00, 1.500e+00, 1.440e+02, 3.000e+00, 3.000e+02, 1.000e-02,
       9.000e+00, 1.524e+01])

In [17]:
df['hair_growth'].describe()

count     76.000000
mean      29.684079
std       43.012832
min        0.010000
25%       10.000000
50%       15.000000
75%       30.000000
max      300.000000
Name: hair_growth, dtype: float64

In [18]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

15.0

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

### iClicker Question #3

Which do you think is closest the actual area every living person would occupy (in km²)?

- A) 100
- B) 1000
- C) 10,000
- D) 100,000

In [19]:
df['crammed'].unique()

array(['6000', '10000', '1000000', '0.2', '12', '100000', '2500', '20',
       '1000', '5', '100', '400', '145', '100 000', '1x10^10 km^2', '1',
       '500', '50', '100000000', '54', '900', '10', '100,000', '10000000',
       '365 km ', '600000000', '5000', '290,000,000', '7.5 x 10^6 km ',
       '7200000', '5,000', '1500', '1000 km^2', '500000', '1400', '550',
       '130', '0.01', '700000', '3 million km^2', '20000', '700km²',
       '4000', '2723000000', '285,750 (km^2)', '1500km^2'], dtype=object)

In [20]:
# standardize crammed column
def standardize_crammed(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of inclded unit cases
    string = string.replace("(km^2)", "")
    string = string.replace("km^2", "")
    string = string.replace("km²", "")
    string = string.replace("km", "")

    string = string.strip()
    
    # take care of scientific notation / word cases
    string = string.replace("x10^10", "0000000000")
    string = string.replace("7.5 x 10^6", "7500000")
    string = string.replace("x10^2", "00")
    string = string.replace(" million", "000000")
    
    # take care of case with space in middle
    string = string.replace("100 000", "100000")


    # convert to numeric
    output = float(string)
    
    return output

In [21]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

array([6.0000e+03, 1.0000e+04, 1.0000e+06, 2.0000e-01, 1.2000e+01,
       1.0000e+05, 2.5000e+03, 2.0000e+01, 1.0000e+03, 5.0000e+00,
       1.0000e+02, 4.0000e+02, 1.4500e+02, 1.0000e+10, 1.0000e+00,
       5.0000e+02, 5.0000e+01, 1.0000e+08, 5.4000e+01, 9.0000e+02,
       1.0000e+01, 1.0000e+07, 3.6500e+02, 6.0000e+08, 5.0000e+03,
       2.9000e+08, 7.5000e+06, 7.2000e+06, 1.5000e+03, 5.0000e+05,
       1.4000e+03, 5.5000e+02, 1.3000e+02, 1.0000e-02, 7.0000e+05,
       3.0000e+06, 2.0000e+04, 7.0000e+02, 4.0000e+03, 2.7230e+09,
       2.8575e+05])

In [22]:
# check to see how commas are being handled
df.dtypes

timestamp        object
height_ind      float64
height_class    float64
hair_growth     float64
crammed         float64
SAN_NYC          object
dtype: object

In [23]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

5000.0

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [24]:
df['SAN_NYC'].unique()

array(['lots', '200', '22', '90', '365', '213', '30', '25', '400', '500',
       '60 days', '1 year', '5', '100', '180', '65', '10^3', '600',
       '75 days', '60', '20', '21', '12', '2000', '50', '35', '10', '38',
       '70', '80', '30 days', '7', '488', '36', '680', '50 days', '300',
       '45 days', '6', '38 days', '5 days', '40', '4 months', '350 days'],
      dtype=object)

In [25]:
# standardize distance column
def standardize_distance(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # remove units
    string = string.replace("days", "")
    
    # take care of non-days answers
    string = string.replace("4 months", "120")
    string = string.replace("1 year", "365")
    string = string.replace("lots", "NaN")
    
    # take care of scientific notation 
    string = string.replace("10^3", "10000")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [26]:
# apply function across values in crammed columns
df["SAN_NYC"] = df["SAN_NYC"].apply(standardize_distance)
df["SAN_NYC"].unique()

array([     nan, 2.00e+02, 2.20e+01, 9.00e+01, 3.65e+02, 2.13e+02,
       3.00e+01, 2.50e+01, 4.00e+02, 5.00e+02, 6.00e+01, 5.00e+00,
       1.00e+02, 1.80e+02, 6.50e+01, 1.00e+04, 6.00e+02, 7.50e+01,
       2.00e+01, 2.10e+01, 1.20e+01, 2.00e+03, 5.00e+01, 3.50e+01,
       1.00e+01, 3.80e+01, 7.00e+01, 8.00e+01, 7.00e+00, 4.88e+02,
       3.60e+01, 6.80e+02, 3.00e+02, 4.50e+01, 6.00e+00, 4.00e+01,
       1.20e+02, 3.50e+02])

In [27]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

75.0

## Take Home Messages

1. Look at your data! (People behave unexpectedly.)
2. Get your data into a tidy data format.
3. Fix (Standardize) your data - or, better yet, collect better data!
4. Use your intuition/knowledge/information out there to consider if values make sense.